In [103]:
# Import Image from wand.image module
from wand.image import Image

# Read image using Image() function
with Image(filename ="lenna.png") as img:

	# Generate noise image using noise() function
	img.noise("poisson", attenuate = 0.9)
	img.save(filename ="lenna1.png")


In [104]:
##PSNR (Peak Signal-to-Noise Ratio:
#apt install imagemagick
#pip install opencv-python

import math
import numpy as np
import cv2

img_orig = cv2.imread("lenna.png")
img_compress = cv2.imread("lenna1.png") 


def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


print("PSNR is {} dB".format(calculate_psnr(img_orig, img_compress)))

PSNR is 9.280369137053357 dB


In [105]:
#SSIM (Structural Similarity Index)

import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

img_orig = cv2.imread("lenna.png")
img_compress = cv2.imread("lenna1.png") 

print("SSIM map mean: {}".format(ssim(img_orig, img_compress)))
print("SSIM value: {}".format(calculate_ssim(img_orig, img_compress)))




SSIM map mean: 0.0438623105763882
SSIM value: 0.0438623105763882
